In [1]:
# from extract_feature import *
from mmpretrain import get_model
import os
import mmengine
device = 'cuda:0'
config = "/data2/hongn/sapiens/pretrain/configs/sapiens_mae/humans_300m_test/mae_sapiens_0.3b-p16_8xb512-coslr-1600e_humans_300m_test.py"
checkpoint = "/data2/hongn/sapiens/pretrain/checkpoints/sapiens_0.3b/sapiens_0.3b_epoch_1600_clean.pth"
input = "/data2/hongn/sapiens/pose/demo/data/itw_videos"
output_root = "/data2/hongn/sapiens/outputs/ados2_pretrain"

if output_root:
    mmengine.mkdir_or_exist(output_root)
    output_file = os.path.join(output_root, os.path.basename(input))

# inferencer = FeatureExtractor(model=config, pretrained=checkpoint, device='cuda', backbone=dict(out_indices=(0, 1, 2, 3, 4)))
inferencer = get_model(model=config, pretrained=True, device=device)

/data2/hongn/miniconda3/envs/sapiens_lite/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data2/hongn/sapiens/cv/mmcv/cnn/bricks/transformer.py:38: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv`` rather than ``mmcv-lite`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '
/data2/hongn/miniconda3/envs/sapiens_lite/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
import torch
dummy_video = torch.randn(1, 3, 1024, 1024).to(device)
feats = inferencer.extract_feat(dummy_video)
feats[0].shape

torch.Size([1, 4097, 1024])

In [3]:
import torch
torch.diag(torch.ones(4), 0)

/data2/hongn/miniconda3/envs/sapiens_lite/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [1]:
import numpy as np

def tridiag(a, b, c, k1=-1, k2=0, k3=1):
    return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)

a = [1, 1]; b = [1, 1, 1]; c = [1, 1]
A = tridiag(a, b, c)
A

array([[1, 1, 0],
       [1, 1, 1],
       [0, 1, 1]])

In [16]:
A = torch.diag(torch.ones(4), 0) + torch.ones(4,1)  #+ torch.diag(torch.ones(3), 1) + torch.diag(torch.ones(3), -1)
A 

tensor([[2., 1., 1., 1.],
        [1., 2., 1., 1.],
        [1., 1., 2., 1.],
        [1., 1., 1., 2.]])

In [ ]:
def arrow(a, b, c, k1=-1, k2=0, k3=1):
    return np.diag(a-1, k1) + np.diag(b, k2) + np.diag(c, k3)

tensor([[1., 1., 1., 1.]])

In [17]:
np.ones(3)[np.newaxis,:]

array([[1., 1., 1.]])

In [ ]:
def arrow_matrix(size = 8, n = 2):
    dim = size - 1
    ret = torch.diag(torch.ones(dim), 0)
    for j in range(n):
        for i in range(dim-n-j, dim-j):
            ret = ret + torch.diag(torch.ones(i), dim-i) + torch.diag(torch.ones(i), i-dim)
    row = torch.ones(dim, 1)
    col = torch.ones(1, size)

    return torch.concat((col,torch.concat((row, ret), axis=1)), axis=0)

A = arrow_matrix(size = 8, n = 1)
A

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 0., 1., 1., 1., 0., 0., 0.],
        [1., 0., 0., 1., 1., 1., 0., 0.],
        [1., 0., 0., 0., 1., 1., 1., 0.],
        [1., 0., 0., 0., 0., 1., 1., 1.],
        [1., 0., 0., 0., 0., 0., 1., 1.]])

In [33]:
row = torch.ones(7, 1)
col = torch.ones(1, 8)
B = torch.concat((row, A), axis=1)
torch.concat((col, B), axis=0)

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 0., 1., 1., 1., 0., 0., 0.],
        [1., 0., 0., 1., 1., 1., 0., 0.],
        [1., 0., 0., 0., 1., 1., 1., 0.],
        [1., 0., 0., 0., 0., 1., 1., 1.],
        [1., 0., 0., 0., 0., 0., 1., 1.]])

In [28]:
row

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [87]:
dim = 10
n = 1
img_len=5
ret = torch.diag(torch.ones(dim), 0)
# for i in range(dim-n, dim):
#     ret = ret + torch.diag(torch.ones(i), dim-i) + torch.diag(torch.ones(i), i-dim)

ret = ret + torch.diag(torch.ones(9-5), 10-9-7)
ret

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 1.]])

In [ ]:
def zebra_matrix(dim = 6, n = 2):
    ret = torch.diag(torch.ones(dim), 0)
    for i in range(dim-n, dim):
        ret = ret + torch.diag(torch.ones(i), dim-i) + torch.diag(torch.ones(i), i-dim)
    return ret

In [24]:
ndiag_matrix(5,3)

tensor([[1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1.]])

In [ ]:
from einops import rearrange
visual_mask = torch.tensor([[0, 1, 1, 1]]).bool()
motion_mask = torch.tensor([[1, 1, 1,1]]).bool() 

attn_mask = rearrange(visual_mask, 'b i -> b 1 i 1') * rearrange(motion_mask, 'b j -> b 1 1 j')

attn_mask

tensor([[[[False, False, False, False],
          [ True,  True,  True,  True],
          [ True,  True,  True,  True],
          [ True,  True,  True,  True]]]])

In [100]:
x = ndiag_matrix(5,1).unsqueeze(0).repeat(3, 1, 1).unsqueeze(1)


In [102]:
x[2][0]

tensor([[1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [0., 1., 1., 1., 0.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.]])

In [35]:
visual_mask

tensor([[True, True, True, True]])

In [30]:
attn_mask[0][0]

tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])

In [47]:
a = torch.tensor([[1,1,0,0,0]]).bool()
b = torch.tensor([[1,1,0,0]]).bool()

In [48]:
attn_mask = rearrange(a, 'b i -> b 1 i 1') * rearrange(b, 'b j -> b 1 1 j')
attn_mask

tensor([[[[ True,  True, False, False],
          [ True,  True, False, False],
          [False, False, False, False],
          [False, False, False, False],
          [False, False, False, False]]]])